In [1]:
import os
import pandas as pd
import numpy as np
import csv
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from collections import Counter
from string import punctuation
import nltk
from string import punctuation
from operator import itemgetter
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import NMF, LatentDirichletAllocation
from nltk.tokenize import sent_tokenize, word_tokenize
ps = PorterStemmer()
import string 
from collections import Counter
from nltk import word_tokenize, pos_tag, pos_tag_sents
import matplotlib.pyplot as plt

In [2]:
os.chdir("E:\\Consultancy\\Crayon")
df = DataFrame.from_csv("100K_Restaurants_reviews.tsv", sep="\t")

In [178]:
os.chdir("E:\\Consultancy\\Crayon")
df = DataFrame.from_csv("100K_Restaurants_reviews.tsv", sep="\t")
df['POS'] = pos_tag_sents(df['ReviewTxt'].apply(word_tokenize).tolist())
df['noun_words'] = np.nan
df['noun_words_sentec'] = np.nan
df = df.astype('object')

In [179]:
for j in range(len(df)):
    tagged = df.iloc[j,9]
    noun_words = []
    for item in tagged:
        if item[1] == 'N' or item[1] == 'NN' or item[1] == 'NNP' or item[1] == 'NNS' or item[1] == 'NNPS':
            noun_words.append(item[0])
    df.iloc[j,10] = noun_words

In [195]:
tf

<99999x156 sparse matrix of type '<class 'numpy.int64'>'
	with 877463 stored elements in Compressed Sparse Row format>

In [181]:
for j in range(len(df)):
    temp = ' '.join(word for word in df.iloc[j,10])
    df.iloc[j,11] = temp

In [193]:
no_features = 1000 #top 1000 features are selected
tf_vectorizer = CountVectorizer(max_df=0.95,min_df=0.02,stop_words='english')
tf = tf_vectorizer.fit_transform(df['noun_words_sentec'])
tf_feature_names = tf_vectorizer.get_feature_names()

In [143]:
lda_model = LatentDirichletAllocation(n_topics=20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(tf)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [201]:
# Define Search Param
search_params = {'n_components': [5, 10, 15, 20, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(tf)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20.

GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_components': [5, 10, 15, 20, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [202]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(tf))

Best Model's Params:  {'learning_decay': 0.7, 'n_components': 5}
Best Log Likelihood Score:  -1742709.36275
Model Perplexity:  97.2563100719


In [203]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(tf)


In [204]:
#Topic distribution over the review text
Topic_distribution = DataFrame(data=lda_output)
Topic_distribution.head()
Topic_distribution.to_csv("E:\\Consultancy\\Crayon\\Topic_distribution.csv")

In [214]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = tf_vectorizer.get_feature_names()

# View
df_topic_keywords.to_csv("E:\\Consultancy\\Crayon\\Topic_words_value.csv")

In [207]:
# choose the best parameter based on this. It should be maximum
# Get Log Likelyhoods from Grid Search Output

n_topics = [5, 10, 15, 20, 30]
log_likelyhoods_5 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ if gscore.parameters['learning_decay']==0.5]
log_likelyhoods_7 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ if gscore.parameters['learning_decay']==0.7]
log_likelyhoods_9 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ if gscore.parameters['learning_decay']==0.9]

# Show graph

plt.figure(figsize=(12, 8))
plt.plot(n_topics, log_likelyhoods_5, label='0.5')
plt.plot(n_topics, log_likelyhoods_7, label='0.7')
plt.plot(n_topics, log_likelyhoods_9, label='0.9')
plt.title("Choosing Optimal LDA Model")
plt.xlabel("Num Topics")
plt.ylabel("Log Likelyhood Scores")
plt.legend(title='Learning decay', loc='best')
plt.savefig('E:\\Consultancy\\Crayon\\comparision.png', dpi=300)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [208]:
# Show top n keywords for each topic
def show_topics(vectorizer=tf_vectorizer, lda_model=lda_model, n_words=100):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=tf_vectorizer, lda_model=best_lda_model, n_words=100)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords.to_csv("E:\\Consultancy\\Crayon\\Topic_words.csv")

In [249]:
data =  df['RestaurantId'].to_frame()
data.to_csv("E:\\Consultancy\\Crayon\\ddummy.csv")

In [253]:
Final_topics = pd.DataFrame.from_csv("Topic_distribution.csv")
Restaurant = Final_topics.groupby(['RestaurantId']).mean()
Restaurant.to_csv("E:\\Consultancy\\Crayon\\Restaurant_topics_distribution.csv")
Review = Final_topics.groupby(['ReviewId']).mean()
Review.to_csv("E:\\Consultancy\\Crayon\\User_topics_distribution.csv")

,ReviewId,RestaurantId,Topic1,Topic2,Topic3,Topic4,Topic5
0,7063897,54162,0.028932,0.204383,0.708920,0.028945,0.028820
1,7059469,54162,0.112233,0.015822,0.494437,0.276923,0.100586
2,7050151,54162,0.202774,0.033661,0.696518,0.033712,0.033334
3,7049203,54162,0.028667,0.346743,0.029633,0.566359,0.028597
4,7046773,54162,0.264663,0.018790,0.679602,0.018684,0.018260


In [3]:
df.columns

Index(['RestaurantName', 'RestaurantId', 'City', 'Rating', 'UserName',
       'UserId', 'DateTime', 'UserLink', 'ReviewTxt'],
      dtype='object')

In [5]:
Final_topics = pd.DataFrame.from_csv("Topic_distribution.csv")
User_ID = Final_topics.groupby(['UserId']).mean()
User_ID.to_csv("E:\\Consultancy\\Crayon\\User_topics_distribution.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
